## CTC Decode

In [2]:
!git clone --recursive https://github.com/parlance/ctcdecode.git

Cloning into 'ctcdecode'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 1063 (delta 6), reused 7 (delta 2), pack-reused 1047
Receiving objects: 100% (1063/1063), 763.61 KiB | 1.32 MiB/s, done.
Resolving deltas: 100% (509/509), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 90, done.        
remote: Counting objects: 100% (90/90), done.        
remote: Compressing objects: 100% (64/64), done.        
remote: Total 13672 (delta 41), reused 54 

In [3]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=ccd56c90a970b9c61f0a8f6cac230901669b9fbff742d7d2b81e7ff92d568330
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [4]:
%cd ctcdecode

/content/ctcdecode


In [5]:
!pip install .

Processing /content/ctcdecode
  Created wheel for ctcdecode: filename=ctcdecode-1.0.2-cp36-cp36m-linux_x86_64.whl size=12784436 sha256=4607ba8e3cef3ad8d27c837af90cb2c47dc83b57d18748a75c496ef110050a15
  Stored in directory: /tmp/pip-ephem-wheel-cache-ge4mvdtw/wheels/c3/6c/94/7d57d4f20a87a22ef1722eaad22052b4c435892b55400e5f4e
Successfully built ctcdecode


In [6]:
%cd ..

/content


## Dataset Download from Kaggle

In [15]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!kaggle competitions download -c 11-785-fall-20-homework-3

mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 98.6M/98.6M [00:02<00:00, 34.6MB/s]

 90% 88.0M/98.1M [00:02<00:00, 33.1MB/s]
100% 98.1M/98.1M [00:02<00:00, 41.6MB/s]
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 216MB/s]
  0% 0.00/236k [00:00<?, ?B/s]
100% 236k/236k [00:00<00:00, 77.7MB/s]
  0% 0.00/1.06k [00:00<?, ?B/s]
100% 1.06k/1.06k [00:00<00:00, 1.10MB/s]
 99% 1.56G/1.57G [00:28<00:00, 46.2MB/s]
100% 1.57G/1.57G [00:28<00:00, 58.2MB/s]
  0% 0.00/16.5k [00:00<?, ?B/s]
100% 16.5k/16.5k [00:00<00:00, 17.0MB/s]
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [16]:
!unzip dev.npy.zip
!unzip dev_labels.npy.zip
!unzip train.npy.zip
!unzip train_labels.npy.zip
!unzip test.npy.zip

Archive:  dev.npy.zip
  inflating: dev.npy                 
Archive:  dev_labels.npy.zip
  inflating: dev_labels.npy          
Archive:  train.npy.zip
  inflating: train.npy               
Archive:  train_labels.npy.zip
  inflating: train_labels.npy        
Archive:  test.npy.zip
  inflating: test.npy                


#### Mount Google Drive (in Google Colab)

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Libraries

In [17]:
import time
import os
import csv
from datetime import datetime
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from ctcdecode import CTCBeamDecoder
from torch.nn import CTCLoss
from torch.nn.utils.rnn import *

### Levenshtein Distance Calculation

In [18]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    return (matrix[size_x - 1, size_y - 1])

In [19]:
N_PHONEMES = 41
PHONEME_LIST = [
    " ",
    "SIL",
    "SPN",
    "AA",
    "AE",
    "AH",
    "AO",
    "AW",
    "AY",
    "B",
    "CH",
    "D",
    "DH",
    "EH",
    "ER",
    "EY",
    "F",
    "G",
    "H",
    "IH",
    "IY",
    "JH",
    "K",
    "L",
    "M",
    "N",
    "NG",
    "OW",
    "OY",
    "P",
    "R",
    "S",
    "SH",
    "T",
    "TH",
    "UH",
    "UW",
    "V",
    "W",
    "Y",
    "Z",
    "ZH"
]

PHONEME_MAP = [
    " ",
    ".", #SIL
    "!", #SPN
    "a", #AA
    "A", #AE
    "h", #AH
    "o", #AO
    "w", #AW
    "y", #AY
    "b", #B
    "c", #CH
    "d", #D
    "D", #DH
    "e", #EH
    "r", #ER
    "E", #EY
    "f", #F
    "g", #G
    "H", #H
    "i", #IH 
    "I", #IY
    "j", #JH
    "k", #K
    "l", #L
    "m", #M
    "n", #N
    "N", #NG
    "O", #OW
    "Y", #OY
    "p", #P 
    "R", #R
    "s", #S
    "S", #SH
    "t", #T
    "T", #TH
    "u", #UH
    "U", #UW
    "v", #V
    "W", #W
    "?", #Y
    "z", #Z
    "Z" #ZH
]


## Required Arguments

In [20]:
class Args():
  def __init__(self):
    self.data_dir='./'
    self.weights_dir='/content/drive/My Drive'
    self.model='LSTM'
    self.nhid=384
    self.nlayers=4
    self.lr=1e-3
    self.clip=0.25
    self.epochs=30
    self.batch_size=40
    self.dropout=0.4
    self.dropouth=0.3
    self.dropouti=0.65
    self.seed=11785
    self.cuda = True
    self.log_interval=40
    self.wdecay=1.2e-6
    self.beam_width=60

args = Args()

## Data

In [21]:
train_labels = np.load('train_labels.npy', allow_pickle=True)
dev_labels = np.load('dev_labels.npy', allow_pickle=True)

In [22]:
counters = [0] * 41
def calc_distribution(labels):
  for l in labels:
      for p in l:
          counters[p] += 1
calc_distribution(train_labels)
calc_distribution(dev_labels)
normalized = [c / sum(counters) for c in counters]

In [23]:
PHONEME_DISTRIBUTION = np.array([0.0, *normalized])
PHONEME_DISTRIBUTION

array([0.        , 0.05271646, 0.00067795, 0.01320721, 0.02685725,
       0.09766807, 0.01354033, 0.00577882, 0.01682307, 0.01700335,
       0.00538583, 0.04696975, 0.02956839, 0.02842286, 0.02692916,
       0.01485176, 0.01767896, 0.00835117, 0.02113829, 0.06063651,
       0.03533551, 0.00441121, 0.02543579, 0.03741151, 0.02843624,
       0.06809466, 0.01049339, 0.0123346 , 0.00098399, 0.01806091,
       0.03871557, 0.04448503, 0.00787857, 0.06505541, 0.0062066 ,
       0.0048915 , 0.01229881, 0.01904022, 0.0223758 , 0.00636848,
       0.02704087, 0.00044015])

## Utility Functions

In [24]:
def to_float_tensor(numpy_array):
    return torch.from_numpy(numpy_array).float()

def to_long_tensor(numpy_array):
    return torch.from_numpy(numpy_array).long()

def to_int_tensor(numpy_array):
    return torch.from_numpy(numpy_array).int()

def to_tensor(numpy_array):
    return torch.from_numpy(numpy_array)

def to_np(x):
    return x.data.cpu().numpy()

def to_variable(tensor):
    if torch.cuda.is_available():
        tensor = tensor.cuda()
    return torch.autograd.Variable(tensor)

## Dataset Class

In [25]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        if y is not None:
            self.total_phonemes = sum(len(yi) for yi in y)
        else:
            self.total_phonemes = -1

    def __getitem__(self, idx):
        frames = self.x[idx]
        return to_float_tensor(frames), to_int_tensor(self.y[idx] + 1 if self.y is not None else np.array([-1]))

    def __len__(self):
        return self.x.shape[0]  

In [26]:
def my_collate(batch):
    batch_size = len(batch)
    batch = sorted(batch, key=lambda b: b[0].size(0), reverse=True)
    max_seq_len = batch[0][0].size(0)
    channels = batch[0][0].size(1)
    pack = torch.zeros(max_seq_len, batch_size, channels)
    all_labels = []
    seq_sizes = []
    label_sizes = torch.zeros(batch_size).int()
    for i, (frames, labels) in enumerate(batch):
        seq_size = frames.size(0)
        seq_sizes.append(seq_size)

        label_size = labels.size(0)
        label_sizes[i] = label_size

        pack[:seq_size, i, :] = frames
        all_labels.append(labels)

    return pack, seq_sizes, all_labels, label_sizes

In [27]:
def get_data_loaders(args, use_dev_for_train=False):

    xdev = np.load(args.data_dir + '/dev.npy', allow_pickle=True)
    ydev = np.load(args.data_dir + '/dev_labels.npy', allow_pickle=True)
    if use_dev_for_train:
        xtrain = np.load(args.data_dir + '/dev.npy', allow_pickle=True)
        ytrain = np.load(args.data_dir + '/dev_labels.npy', allow_pickle=True)
    else:
        xtrain = np.load(args.data_dir + '/train.npy', allow_pickle=True)
        ytrain = np.load(args.data_dir + '/train_labels.npy', allow_pickle=True)

    kwargs = {'num_workers': 3, 'pin_memory': True} if args.cuda else {}
    train_loader = torch.utils.data.DataLoader(
        MyDataset(xtrain, ytrain),
        batch_size=args.batch_size, shuffle=True, collate_fn=my_collate, **kwargs)
    dev_loader = torch.utils.data.DataLoader(
        MyDataset(xdev, ydev),
        batch_size=args.batch_size, shuffle=True, collate_fn=my_collate, **kwargs)

    return train_loader, dev_loader

## Model

In [28]:
class LockedDropout(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, dropout=0.5):
        if dropout == 0 or not self.training:
            return x
        mask = x.data.new(1, x.size(1), x.size(2))
        mask = mask.bernoulli_(1 - dropout)
        mask = Variable(mask, requires_grad=False) / (1 - dropout)
        mask = mask.expand_as(x)
        return mask * x

In [29]:
class SequenceWise(nn.Module):
    def __init__(self, module):
        super(SequenceWise, self).__init__()
        self.module = module

    def forward(self, x):
        t, n = x.size(0), x.size(1)
        x = x.view(t * n, -1)
        x = self.module(x)
        x = x.view(t, n, -1)
        return x

    def __repr__(self):
        tmpstr = self.__class__.__name__ + ' (\n'
        tmpstr += self.module.__repr__()
        tmpstr += ')'
        return tmpstr

In [30]:
class BnLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, rnn_type=nn.LSTM, batch_norm=False):
        super(BnLSTM, self).__init__()
        self.batch_norm = SequenceWise(nn.BatchNorm1d(hidden_size * 2)) if batch_norm else None
        self.rnn = rnn_type(input_size, hidden_size, bidirectional=True)

    def forward(self, x, seq_sizes):
        out = pack_padded_sequence(x, seq_sizes)
        out, h = self.rnn(out)
        out, _ = pad_packed_sequence(out)

        if self.batch_norm is not None:
            out = self.batch_norm(out)
        return out, h

In [31]:
class MyModel(nn.Module):
    def __init__(self, nhid, nlayers, dropout=0, dropouth=0, dropouti=0, n_input=13, n_output=42):
        super(MyModel, self).__init__()
        self.nhid = nhid
        self.nlayers = nlayers
        self.lockdrop = LockedDropout()
        self.dropout = dropout
        self.dropouth = dropouth
        self.dropouti = dropouti

        self.cnns = torch.nn.Sequential(
            nn.BatchNorm1d(n_input),
            nn.Conv1d(n_input, nhid, 3, padding=1),
            nn.BatchNorm1d(nhid),
            nn.Hardtanh(inplace=True)
        )

        rnns = [BnLSTM(nhid, nhid)]
        for i in range(nlayers - 1):
            rnns.append(BnLSTM(nhid * 2, nhid))

        self.rnns = torch.nn.ModuleList(rnns)
        self.linear_out = nn.Linear(nhid * 2, n_output)
        self.init_weights()
        self.hs = []

    def forward(self, data, seq_sizes):
        data = data.permute(1, 2, 0)
        data = data.contiguous()
        out = self.cnns(data)
        out = out.permute(2, 0, 1)
        out = self.lockdrop(out, self.dropouti)
        for l, rnn in enumerate(self.rnns):
            out, h = rnn(out, seq_sizes)
            out = self.lockdrop(out, self.dropout if l == self.nlayers - 1 else self.dropouth)

        out = self.linear_out(out)
        return out

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight.data)
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
        
        self.linear_out.bias.data = to_float_tensor(PHONEME_DISTRIBUTION)

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        self.hs = []
        for l in range(self.nlayers):
            n = self.nhid
            self.hs.append((Variable(weight.new(1, bsz, n).zero_()),
                            Variable(weight.new(1, bsz, n).zero_())))

    def repackage_hidden(self, h):
        if type(h) == Variable:
            return Variable(h.data)
        else:
            return tuple(self.repackage_hidden(v) for v in h)

In [32]:
def cer(s1, s2):
    s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')
    return levenshtein(s1, s2)

## Train / Evaluate Functions

In [33]:
def label2str(label):
    return "".join(PHONEME_MAP[i] for i in label)


def decode(output, seq_sizes, beam_width=args.beam_width):
    decoder = CTCBeamDecoder(labels=PHONEME_MAP, blank_id=0, beam_width=beam_width)
    output = torch.transpose(output, 0, 1) 
    probs = F.softmax(output, dim=2).data.cpu()

    output, scores, timesteps, out_seq_len = decoder.decode(probs=probs, seq_lens=torch.IntTensor(seq_sizes))

    decoded = []
    for i in range(output.size(0)):
        chrs = ""
        if out_seq_len[i, 0] != 0:
            chrs = "".join(PHONEME_MAP[o] for o in output[i, 0, :out_seq_len[i, 0]])
        decoded.append(chrs)
    return decoded


def print_score_board(score_board, N=10):
    score_board.sort(key=lambda x: x[0])
    print("\nTop-{}\n".format(N))
    print("\n".join(["CER:\t{:6.4f}\nLabel:\t{}\nMine:\t{}\n".format(i[0], i[1], i[2]) for i in score_board[:N]]))
    print("\nLast-{}\n".format(N))
    print("\n".join(["CER:\t{:6.4f}\nLabel:\t{}\nMine:\t{}\n".format(i[0], i[1], i[2]) for i in score_board[-N:]]))


def evaluate(model, criterion, loader, args, calc_cer=False):
    model.eval()
    total_loss = 0
    total_cer = 0
    score_board = []
    with torch.no_grad():
      for batch, (frames, seq_sizes, labels, label_sizes) in enumerate(loader):
          batch += 1

          if args.cuda:
              frames = frames.cuda()
          frames = Variable(frames)
          output = model(frames, seq_sizes)

          loss = criterion(output,
                          Variable(torch.cat(labels).int()),
                          Variable(torch.IntTensor(seq_sizes)),
                          Variable(label_sizes))
          total_loss += loss.item()

          if (batch == 1) or calc_cer:
              decoded = decode(output, seq_sizes)
              label_str = [label2str(label) for label in labels]
              for l, m in zip(label_str, decoded):
                  e = cer(l, m)
                  total_cer += e
                  score_board.append([e * 100.0 / len(l), l, m])

    print_score_board(score_board)

    total_phonemes = loader.dataset.total_phonemes
    return total_loss / (len(loader) * args.batch_size), total_loss / total_phonemes, total_cer * 100.0 / total_phonemes


def train(epoch, model, optimizer, criterion, loader, args):
    model.train()
    sum_loss, sum_phonemes = 0, 0
    start_time = time.time()
    optimizer.zero_grad()

    for batch, (frames, seq_sizes, labels, label_sizes) in enumerate(loader):
        batch += 1

        sum_phonemes += sum(len(l) for l in labels)

        if args.cuda:
            frames = frames.cuda()
        frames = Variable(frames)

        output = model(frames, seq_sizes)

        loss = criterion(output,
                         Variable(torch.cat(labels).int()),
                         Variable(torch.IntTensor(seq_sizes)),
                         Variable(label_sizes))
        loss.backward()

        sum_loss += loss.item()

        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)

        optimizer.step()
        optimizer.zero_grad()

        if batch % args.log_interval == 0:
            elapsed = time.time() - start_time
            avg_loss = sum_loss / sum_phonemes
            print(
                '| epoch {:3d} | {:5d}/{:5d} batches ({:5.2f}%) | lr {:.2e} | {:3.0f} ms/utter | loss/utter {:5.2f} | loss/phoneme {:5.4f}'
                .format(epoch, batch, len(loader), (100.0 * batch / len(loader)),
                        optimizer.param_groups[0]['lr'],
                        elapsed * 1000.0 / (args.log_interval * args.batch_size),
                        sum_loss / (args.log_interval * args.batch_size),
                        avg_loss))
            sum_loss, sum_phonemes = 0, 0
            start_time = time.time()

## Training

In [37]:
def main(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)

    model = MyModel(args.nhid, args.nlayers, args.dropout, args.dropouth, args.dropouti)
    #model.load_state_dict(torch.load(args.weights_dir+'/HW3P2.w'))
    if args.cuda:
        model.cuda()

    train_loader, dev_loader = get_data_loaders(args)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.wdecay)
    criterion = CTCLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, threshold=0.01, verbose=True)
    for epoch in range(1, args.epochs + 1):
        print(datetime.now())
        epoch_start_time = time.time()
        train(epoch, model, optimizer, criterion, train_loader, args)
        if True:
            val_loss_utter, val_loss_phoneme, val_cer = evaluate(model, criterion, dev_loader, args, calc_cer=True)
            scheduler.step(val_loss_phoneme)
            print('-' * 100)
            print('| end of epoch {:3d} | time: {:5.2f}s | loss/utter {:5.2f} | loss/phoneme {:5.4f} | valid cer {:5.4f}'
                  .format(epoch, (time.time() - epoch_start_time), val_loss_utter, val_loss_phoneme, val_cer))
            print('-' * 100)

            if not os.path.exists(args.weights_dir):
                os.makedirs(args.weights_dir)
            weight_fname = "{}/HW3P2.w".format(args.weights_dir)
            torch.save(model.state_dict(), weight_fname)

In [38]:
args.cuda = torch.cuda.is_available() and args.cuda

In [39]:
main(args)

2020-11-15 17:41:11.787601
| epoch   1 |    40/  551 batches ( 7.26%) | lr 1.00e-03 |  29 ms/utter | loss/utter  0.10 | loss/phoneme 0.0007
| epoch   1 |    80/  551 batches (14.52%) | lr 1.00e-03 |  28 ms/utter | loss/utter  0.09 | loss/phoneme 0.0007


KeyboardInterrupt: ignored

## Test Prediction

In [29]:
def get_test_data_loaders(args):
    print("loading data")
    xtest = np.load(args.data_dir + '/test.npy', allow_pickle=True)
    print("load complete")
    kwargs = {'pin_memory': True} if args.cuda else {}
    test_loader = torch.utils.data.DataLoader(
        MyDataset(xtest, None),
        batch_size=1, shuffle=False, collate_fn=my_collate, **kwargs)
    return test_loader

In [30]:
def predict(args, csv_fpath, weights_fpath):
    model = MyModel(args.nhid, args.nlayers, args.dropout, args.dropouth, args.dropouti)
    model.load_state_dict(torch.load(weights_fpath))
    if args.cuda:
        model.cuda()

    model.eval()
    test_loader = get_test_data_loaders(args)
    with open(csv_fpath, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['id', 'label'])
        writer.writeheader()
        cnt = 0
        with torch.no_grad():
          for batch, (frames, seq_sizes, _, _) in enumerate(test_loader):
              batch += 1

              if args.cuda:
                  frames = frames.cuda()
              frames = Variable(frames)
              output = model(frames, seq_sizes)

              decoded = decode(output, seq_sizes, beam_width=100)
              for s in decoded:
                  writer.writerow({"id": cnt, "label": s})
                  cnt += 1
    print("done")

In [ ]:
predict(args, 'submission.csv', args.weights_dir+'/HW3P2.w')